In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h1 style=color:blue align="center"> Tabular Playground Series - Mar 2021 </h>

### Data Description
- For this competition, you will be predicting a **binary target** based on a number of feature columns given in the data.

- Train set has **300,000 rows** while test set has **200,000 rows**. There are **19 categorical features** from **cat0 - cat18** and **11 continuous features** from **cont0 - cont10** with total of **30 features.**

### Files
- **train.csv** - the training data with the target column
- **test.csv** - the test set; you will be predicting the target for each row in this file (the probability of the binary target)
- **sample_submission.csv** - a sample submission file in the correct format

- **Submissions** are evaluated on **area under the ROC curve** between the predicted probability and the observed target.

### Content:
1. EDA(Exploratory Data Analysis)

   a. Manual EDA
   
   b. Pandas Profiling
   
   c. SweetViz
   
   d. AutoViz
    
2. Outliers
3. Correlation
4. Skewness and Kurtosis
5. Data Visualisation
6. Feature Engineering
7. Model building & Evaluation

<h1 style=color:blue align="left"> 1. Import Required Libraries </h1>

In [ ]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns 

import warnings
warnings.filterwarnings("ignore")

plt.style.use("fivethirtyeight")
sns.set_style("darkgrid")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import roc_auc_score
import category_encoders as ce
from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.decomposition import PCA

import optuna

<h1 style=color:blue align="left"> 2. Load Data </h1>

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2021/test.csv")
#submission = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv")
sub = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv")

In [ ]:
train_original = train.copy()
test_original = test.copy()

In [ ]:
display (train.head())
display (test.head())
display (sub.head())

In [ ]:
display(train.shape)
display(test.shape)

In [ ]:
display(train.info())
display(test.info())

In [ ]:
print("Columns in Train: \n\n", train.columns.tolist())
print("\nColumns in Test: \n\n", test.columns.tolist())

<h1 style=color:blue align="left"> 3. EDA(Exploratory Data Analysis) </h1>

<h2 style=color:green align="left"> i) Pandas Profiling </h2>

In [ ]:
import pandas_profiling as pp
pp.ProfileReport(train)

In [ ]:
print("--------DETAILS OF TRAINING AND TESTING DATA SIZE---------")
print("Number of instances in train data: ",len(train))
print("Number of instances in test data: ",len(test))
print("There are {} feature columns".format(test.shape[1]))

<h2 style=color:green align="left"> ii) AutoViz </h2>

In [ ]:
!pip install AutoViz
!pip install xlrd

In [ ]:
from autoviz.AutoViz_Class import AutoViz_Class

AV = AutoViz_Class()
dftc = AV.AutoViz(filename='', 
                  sep ='' , 
                  depVar ='target', 
                  dfte = train, 
                  header = 0, 
                  verbose = 1, 
                  lowess = False, 
                  chart_format ='png', 
                  max_rows_analyzed = 300000, 
                  max_cols_analyzed = 30
)

<h2 style=color:green align="left"> ii) Missing Values </h2>

In [ ]:
display(train.isnull().sum())
display(test.isnull().sum())
print(train.isnull().sum().sum())
print(test.isnull().sum().sum())

<h2 style=color:green align="left"> 1) target </h2>

In [ ]:
train['target'].value_counts().to_frame()

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x='target', data=train)
plt.show()

In [ ]:
def plot_categorical(cat, train=train):
    ax = plt.subplots(figsize=(12, 6))
    sns.set_style('whitegrid')
    sns.countplot(x=cat, data=train);
    plt.ylabel('No. of Observations', size=20);
    plt.xlabel(cat+' Count', size=20);
    plt.show()
    
def plot_cat_response(cat, train=train):
    ax = plt.subplots(figsize=(12, 8))
    sns.set_style('whitegrid')
    sns.countplot(x=cat, hue='target', data=train);
    plt.show()

<h2 style=color:green align="left"> 2) cat0 </h2>

In [ ]:
plot_categorical('cat0')

In [ ]:
plot_cat_response('cat0')

<h2 style=color:green align="left"> 3) cat1 </h2>

In [ ]:
plot_categorical('cat1')

In [ ]:
plot_cat_response('cat1')

<h2 style=color:green align="left"> 4) cat2 </h2>

In [ ]:
plot_categorical('cat2')

In [ ]:
plot_cat_response('cat2')

<h2 style=color:green align="left"> 5) cat3 </h2>

In [ ]:
plot_categorical('cat3')

In [ ]:
plot_cat_response('cat3')

<h2 style=color:green align="left"> 6) cat4 </h2>

In [ ]:
plot_categorical('cat4')

In [ ]:
plot_cat_response('cat4')

<h2 style=color:green align="left"> 7) cat5 </h2>

In [ ]:
plot_categorical('cat5')

In [ ]:
plot_cat_response('cat5')

<h2 style=color:green align="left"> 8) cat6 </h2>

In [ ]:
plot_categorical('cat6')

In [ ]:
plot_cat_response('cat6')

<h2 style=color:green align="left"> 9) cat7 </h2>

In [ ]:
plot_categorical('cat7')

In [ ]:
plot_cat_response('cat7')

<h2 style=color:green align="left"> 10) cat8 </h2>

In [ ]:
plot_categorical('cat8')

In [ ]:
plot_cat_response('cat8')

<h2 style=color:green align="left"> 11) cat9 </h2>

In [ ]:
plot_categorical('cat9')

In [ ]:
plot_cat_response('cat9')

<h2 style=color:green align="left"> 12) cat10 </h2>

In [ ]:
plot_categorical('cat10')

In [ ]:
plot_cat_response('cat10')

<h2 style=color:green align="left"> 13) cat11 </h2>

In [ ]:
plot_categorical('cat11')

In [ ]:
plot_cat_response('cat11')

<h2 style=color:green align="left"> 14) cat12 </h2>

In [ ]:
plot_categorical('cat12')

In [ ]:
plot_cat_response('cat12')

<h2 style=color:green align="left"> 15) cat13 </h2>

In [ ]:
plot_categorical('cat13')

In [ ]:
plot_cat_response('cat13')

<h2 style=color:green align="left"> 16) cat14 </h2>

In [ ]:
plot_categorical('cat14')

In [ ]:
plot_cat_response('cat14')

<h2 style=color:green align="left"> 17) cat14 </h2>

In [ ]:
plot_categorical('cat15')

In [ ]:
plot_cat_response('cat15')

<h2 style=color:green align="left"> 18) cat16 </h2>

In [ ]:
plot_categorical('cat16')

In [ ]:
plot_cat_response('cat16')

<h2 style=color:green align="left"> 18) cat17 </h2>

In [ ]:
plot_categorical('cat17')

In [ ]:
plot_cat_response('cat17')

<h2 style=color:green align="left"> 19) cat18 </h2>

In [ ]:
plot_categorical('cat18')

In [ ]:
plot_cat_response('cat18')

<h2 style=color:green align="left"> ii) Correlation between Features </h2>

In [ ]:
train.corr()['target'].sort_values(ascending=False)[:25]

In [ ]:
a = train.drop("target", axis=1)
a.corrwith(train.target).plot(kind='bar', grid=True, figsize=(15,10), color='salmon')
plt.title("Correlation with target", size=20)
plt.xticks(size=15, rotation=90)
plt.yticks(size=15)
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(train.corr(), annot=True, cbar=False, fmt='.1g', linewidth=0.3);

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(train.corr(method='pearson'), annot=True, cbar=False, fmt='.1g', linewidth=0.3)
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(train.corr(method='spearman'), annot=True, cbar=False, fmt='.1g', linewidth=0.3)
plt.show()

<h2 style=color:green align="left"> iii) Drop Unwanted Columns </h2>

In [ ]:
#y = train['target']
train.drop(['id','target'], axis=1, inplace=True)
test.drop(['id'], axis=1, inplace=True)

<h2 style=color:green align="left"> iv) Outliers </h2>

In [ ]:
plt.figure(figsize=(10,8))
sns.boxplot(data=train, orient="h", palette="Set2")
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
sns.boxplot(data=test, orient="h", palette="Set2"); # colon used to avoid plt.show()

<h2 style=color:green align="left"> v) Skewness and Kurtsis </h2>

In [ ]:
train.skew().sort_values(ascending=True)[:20]

In [ ]:
test.skew().sort_values(ascending=True)[:20]

<h2 style=color:green align="left"> vi) Data Visualisation </h2>

- Univariate Analysis

- Bivariate Analysis

- Multivariate Analysis

### Univariate Analysis

In [ ]:
!pip install sweetviz

In [ ]:
import sweetviz as sv
my_report = sv.analyze(train)
#my_report.show_html()
my_report.show_notebook(w="100%", h="full")

### Multivariate Analysis

In [ ]:
sns.pairplot(data=train)
plt.show()

In [ ]:
sns.pairplot(data=test)
plt.show()

<h1 style=color:blue align="left"> 4. Feature Engineering </h1>

In [ ]:
def preprocess(df):
    categorical_cols = [c for c in df.columns if 'cat' in c]
    numerical_cols = [c for c in df.columns if 'cat' not in c]
    
    onehot_encoded_df = pd.get_dummies(df[categorical_cols])
    numerical_df = df[numerical_cols]
    
    return pd.concat([numerical_df, onehot_encoded_df], axis=1)

In [ ]:
train_df = pd.read_csv('../input/tabular-playground-series-mar-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-mar-2021/test.csv')

In [ ]:
train_size = train_df.shape[0]
test_size = test_df.shape[0]
all_data = pd.concat([train_df, test_df])

In [ ]:
all_data = preprocess(all_data)
train_data = all_data[:train_size]
test_data = all_data[train_size:].drop(columns=['target'])

In [ ]:
y = train_data.target.values
X = train_data.drop(columns=['id', 'target'])
X_ = test_data.drop(columns='id')

<h1 style=color:blue align="left"> 5. Model Building and Evaluation </h1>

In [ ]:
params = {
    'lambda': 0.0001,
    'learning_rate': 0.007930236488607134,
    'max_bin': 270,
    'max_depth': 98,
    'metric': 'auc',
    'min_data_in_leaf': 60,
    'n_estimators': 20000,
    'num_leaves': 263,
    'objective': 'binary',
    'sub_feature': 0.2098021977637481
}

In [ ]:
folds = KFold(n_splits = 50, shuffle=True)
oof = np.zeros(X.shape[0])
predictions = np.zeros(X_.shape[0])

In [ ]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
    print("Fold {}".format(fold_))
    X_train = X.iloc[trn_idx]
    y_train = y[trn_idx]
    X_test = X.iloc[val_idx]
    y_test = y[val_idx]
    clf = lgb.LGBMClassifier(**params, random_state=42)
    clf.fit(X_train, y_train, eval_set=[(X_train, y_train),(X_test, y_test)],
        eval_metric='auc', early_stopping_rounds=250, verbose=250  )
    predictions += clf.predict_proba(X_, num_iteration=clf.best_iteration_)[:,1] / folds.n_splits

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-mar-2021/sample_submission.csv')
submission = pd.concat([submission, pd.DataFrame(predictions)], axis=1).drop(columns='target')
submission.columns = ['id', 'target']
submission.to_csv('submission.csv', index=False)